# Predicción de Litologías con Aprendizaje Automático

Este proyecto tiene como objetivo predecir litologías a partir de datos geológicos usando técnicas de imputación, transformación de señales y modelos de clasificación. A continuación se detallan los pasos realizados y se presenta una demostración funcional del código.


Importación de Librerías

In [19]:
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import ExtraTreesRegressor
from xgboost import XGBClassifier

import pywt
import pickle
import precond
import wavelet_transform
import feature_augmentation
import validation
import imputation

from datetime import datetime


Carga del Dataset y Estadísticas

In [14]:
start_time = datetime.now()

train = pd.read_csv('train.csv', sep=';')
train_bkp = train.copy()
train.describe()


,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,CALI,RSHA,RMED,RDEP,RHOB,GR,...,ROP,DTS,DCAL,DRHO,MUDWEIGHT,RMIC,ROPA,RXO,FORCE_2020_LITHOFACIES_LITHOLOGY,FORCE_2020_LITHOFACIES_CONFIDENCE
count,1.170511e+06,1.159736e+06,1.159736e+06,1.159736e+06,1.082634e+06,630650.000000,1.131518e+06,1.159496e+06,1.009242e+06,1.170511e+06,...,535071.000000,174613.000000,298833.000000,987857.000000,316151.000000,176160.000000,192325.000000,327427.000000,1.170511e+06,1.170332e+06
mean,2.184087e+03,4.856310e+05,6.681276e+06,-2.138527e+03,1.318568e+01,10.694664,4.986978e+00,1.069103e+01,2.284987e+00,7.091370e+01,...,137.367965,204.655019,1.223849,0.012196,1.216329,7.796809,23.505069,-95.779496,6.138598e+04,1.164258e+00
std,9.971821e+02,3.455641e+04,1.281524e+05,9.709426e+02,3.798907e+00,100.642597,5.467269e+01,1.139480e+02,2.532835e-01,3.423149e+01,...,1539.383558,71.068461,54.372859,7.477798,10.180834,89.741807,22.632800,348.980030,1.389170e+04,4.451180e-01
min,1.360860e+02,4.268988e+05,6.406641e+06,-5.395563e+03,2.344000e+00,0.000100,-8.418695e-03,3.170056e-02,7.209712e-01,1.092843e-01,...,-0.117977,69.163177,-12.215459,-7429.338867,0.125818,0.056586,-999.250122,-999.900024,3.000000e+04,1.000000e+00
25%,1.418597e+03,4.547996e+05,6.591327e+06,-2.811502e+03,9.429712e+00,0.854120,9.140862e-01,9.102396e-01,2.092203e+00,4.762722e+01,...,5.628000,155.936707,0.148438,-0.009253,0.143792,0.891272,11.281895,0.687681,6.500000e+04,1.000000e+00
50%,2.076605e+03,4.769203e+05,6.737311e+06,-2.042785e+03,1.255575e+01,1.399020,1.443584e+00,1.439000e+00,2.321228e+00,6.836763e+01,...,17.799999,188.200653,0.557320,0.001752,0.155774,1.967156,20.131153,1.366672,6.500000e+04,1.000000e+00
75%,2.864393e+03,5.201532e+05,6.784886e+06,-1.391866e+03,1.671075e+01,3.099348,2.680930e+00,2.557220e+00,2.488580e+00,8.903551e+01,...,34.812794,224.645081,1.299655,0.021702,1.168307,5.084564,31.138481,3.420790,6.500000e+04,1.000000e+00
max,5.436632e+03,5.726328e+05,6.856661e+06,-1.110860e+02,2.827900e+01,2193.904541,1.988616e+03,1.999887e+03,3.457820e+00,1.076964e+03,...,47015.125000,676.578125,10011.422852,2.836938,185.730927,10000.000000,742.797852,35930.671875,9.900000e+04,3.000000e+00


## Preprocesamiento e Imputación

Se realiza el preprocesamiento y la imputación de datos faltantes. Se codifican las variables categóricas y se seleccionan las características relevantes.


In [15]:
# Preprocesamiento
# train = precond.precond_train(train)

# Codificación
# le = LabelEncoder()
# le.fit(train['GROUP'])
# train['GROUP'] = le.transform(train['GROUP'])
# pickle.dump(le, open('labelencoder.pkl', 'wb'))

# Reducción de columnas
# train_imp = train.drop(['WELL','FORCE_2020_LITHOFACIES_LITHOLOGY','FORCE_2020_LITHOFACIES_CONFIDENCE'], axis=1)
# train_imp, model_imp_list = imputation.imputer_train(train_imp, "train_imp.csv")
# train_imp['WELL'] = train['WELL']


## Selección de Características, Transformaciones y Aumento

Se seleccionan variables clave, se aplican transformaciones wavelet y se realiza el aumento de características.


In [20]:
features = ['WELL','X_LOC','Y_LOC','Z_LOC','RDEP','GROUP','CALI','GR','RHOB','NPHI','PEF','DTC','SP','DRHO']
# train_imp = train_imp[features]
# train_imp['FORCE_2020_LITHOFACIES_LITHOLOGY'] = train['FORCE_2020_LITHOFACIES_LITHOLOGY']
# train_imp = wavelet_transform.wavelet_transform(train_imp)

# Eliminación del efecto hombro
# train_imp.loc[train_imp['FORCE_2020_LITHOFACIES_LITHOLOGY'].shift(-1) != train_imp['FORCE_2020_LITHOFACIES_LITHOLOGY'].shift(1), 'FORCE_2020_LITHOFACIES_LITHOLOGY'] = np.nan
# train_imp.dropna(subset=['FORCE_2020_LITHOFACIES_LITHOLOGY'], inplace=True)

# Aumento
# train_imp_aug = train_imp.drop(['WELL','FORCE_2020_LITHOFACIES_LITHOLOGY'], axis=1)
# train_imp_aug = feature_augmentation.feat_aug(train_imp_aug, train_imp['WELL'], train_imp['Z_LOC'])
# train_imp_aug = feature_augmentation.poly_feat(train_imp_aug)


## Entrenamiento del Modelo

Se entrena un modelo de clasificación XGBoost utilizando una división entrenamiento/prueba del conjunto de datos.


In [17]:
# X = train_imp_aug
# y = train_imp['FORCE_2020_LITHOFACIES_LITHOLOGY']
# X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=0)

# model = XGBClassifier(max_depth=3, learning_rate=0.1, n_estimators=300, random_state=0)
# model.fit(X_train, y_train)
# print("Model trained")

# accuracy_split_test, pen_rel_split_test = validation.validation(X_test, y_test, train_imp, model, 'penalty_matrix.npy') 
# pickle.dump(model, open('model.pkl', 'wb'))


In [27]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Suponiendo que 'train' es tu DataFrame original
train_features = train.drop(columns=['WELL'])  # O ajusta a tus necesidades
num_cols = train_features.select_dtypes(include='number').columns

# Imputación solo sobre columnas numéricas
imputer = IterativeImputer(random_state=0)
train_imp_num = pd.DataFrame(imputer.fit_transform(train_features[num_cols]), columns=num_cols)

# Combina columnas imputadas con las no numéricas que no necesitan imputación
non_num_cols = train_features.drop(columns=num_cols).reset_index(drop=True)
train_imp = pd.concat([train_imp_num, non_num_cols], axis=1)

# Si quieres guardar para futuros usos:
train_imp.to_csv('train_imp.csv', sep=';', index=False)


## Predicción en Conjunto de Prueba Cerrado

Se carga el conjunto cerrado, se preprocesa y se predicen las litologías utilizando el modelo entrenado.


In [ ]:
# ==================== PREDICCIÓN ====================
model = pickle.load(open('model.pkl', 'rb'))
prediction = model.predict(test_imp_aug)

# Refinamiento
test['PREDICTION'] = prediction
test.loc[test['PREDICTION'].shift(-2) == test['PREDICTION'].shift(1), 'PREDICTION'] = test['PREDICTION'].shift(-2)
test.loc[test['PREDICTION'].shift(-1) == test['PREDICTION'].shift(1), 'PREDICTION'] = test['PREDICTION'].shift(-1)
test['PREDICTION'].iloc[-1] = prediction[-1]

# Guardado CSV
np.savetxt('GIR_TEAM_final_submission.csv', test['PREDICTION'].values, header='lithology', comments='', fmt='%i')

# ==================== GUARDAR GRÁFICA ====================
plt.figure(figsize=(4, 12))
plt.scatter(test['PREDICTION'], test['Z_LOC'], c=test['PREDICTION'], cmap='tab20', s=10)
plt.gca().invert_yaxis()
plt.xlabel("Predicción de litología")
plt.ylabel("Profundidad (Z_LOC)")
plt.title("Predicción de Fases Litológicas")
plt.grid(True)
plt.tight_layout()
plt.savefig("prediction_plot.png")
plt.close()

# ==================== FINAL ====================
end_time = datetime.now()
print('Duración total: {}'.format(end_time - start_time))
print("✅ CSV guardado como: GIR_TEAM_final_submission.csv")
print("✅ Imagen guardada como: prediction_plot.png")